In [ ]:
# install
%%capture
!pip uninstall -y bitsandbytes triton
!pip install triton==2.1.0
!pip install bitsandbytes==0.43.2
!pip install --upgrade unsloth accelerate datasets peft trl sentencepiece protobuf huggingface_hub hf_transfer
!pip install unsloth_zoo
!pip install gradio

In [ ]:
from unsloth import FastLanguageModel
import torch

# load model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit",
    max_seq_length = 2048,
    dtype = torch.float16,
    load_in_4bit = True,
)

# add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0.05,
    bias = "none",
)

In [ ]:
# dataset
from datasets import Dataset

data = [
    {
        "instruction": "How can I manage daily stress?",
        "input": "I feel anxious balancing work and family life.",
        "output": "Try mindfulness meditation for 10 minutes daily and organize tasks with a priority list."
    },
    {
        "instruction": "How to deal with public speaking anxiety?",
        "input": "I get nervous before speaking to a crowd.",
        "output": "Practice breathing exercises and rehearse your speech multiple times."
    },
    {
        "instruction": "Tips for better sleep during stressful periods?",
        "input": "I can't fall asleep easily during exams.",
        "output": "Avoid caffeine before bed, maintain a sleep routine, and do calming activities like reading."
    }
]

dataset = Dataset.from_list(data)

In [ ]:
def format(example):
    return {
        "input": f"{example['instruction']}\n{example['input']}",
        "output": example['output'],
    }

dataset = dataset.map(format)

# tokenize for model
def tokenize(example):
    return tokenizer(
        f"### Instruction:\n{example['input']}\n\n### Response:\n{example['output']}",
        truncation=True,
        padding="max_length",
        max_length=2048,
    )

dataset = dataset.map(tokenize)

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

args = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    warmup_steps = 10,
    max_steps = 100,
    learning_rate = 2e-4,
    fp16 = True,
    logging_steps = 10,
    output_dir = "./mental_health_model",
    optim = "paged_adamw_8bit",
    save_total_limit = 2,
)

# finetune
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    args = args,
    dataset_text_field = None,
    max_seq_length = 2048,
    packing = False,
)

trainer.train()

In [ ]:
# save LoRA adapters + tokenizer
trainer.model.save_pretrained("./mental_health_model")
tokenizer.save_pretrained("./mental_health_model")

In [ ]:
def chat_with_bot():
    print("🧠 Mental Health Chatbot (type 'exit' to quit)\n")

    while True:
        user_input = input("You: ")
        if user_input.lower() in ["exit", "quit"]:
            print("👋 Goodbye!")
            break

        prompt = f"""### Instruction:
{user_input}

### Response:"""

        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=200,
                do_sample=True,
                top_p=0.95,
                temperature=0.7,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        response_text = response.split("### Response:")[-1].strip()
        print(f"Bot: {response_text}\n")


chat_with_bot()

In [ ]:
import gradio as gr

def gradio_chat(user_message):
    prompt = f"""### Instruction:
{user_message}

### Response:"""

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=200,
            do_sample=True,
            top_p=0.95,
            temperature=0.7,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    response_text = response.split("### Response:")[-1].strip()
    return response_text

gr.Interface(
    fn = gradio_chat,
    inputs = gr.Textbox(lines=2, placeholder="Ask me anything about mental health!"),
    outputs = gr.Textbox(),
    title = "🧠 Mental Health Chatbot",
    description = "Ask mental health, wellness, anxiety, stress relief tips.",
    theme = "default",
).launch(share=True)